In [804]:
import numpy as np
import pandas as pd
from itertools import compress
import matplotlib.pyplot as plt
%matplotlib inline
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import abm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [805]:
dat = pd.read_csv('../data_clean/gss18cat.raw', sep='\s+', header=0)
dat.autmean = -dat.autmean
dat.head()

,educ,age,nonwhite,occ17,presmean,autmean
0,19,29,0,2,59.3,0.103448
1,16,29,0,7,35.9,0.083306
2,18,26,0,3,52.4,0.264063
3,12,37,0,12,30.1,0.156604
4,12,30,0,7,35.9,0.083306


In [806]:
n_i = dat.shape[0]
n_j = len(np.unique(dat.occ17))
p_i = 4
p_j = 2
choice = dat.occ17 + 1

# ---- Prepare opp, the opportunity set ----
opp = np.zeros(shape=(n_i, n_j), dtype="bool")
opp[np.arange(n_i), dat.occ17] = True  # people are offered jobs they have!
opp[:, 0] = True                     # Unemployment always offered

In [807]:
# ---- Prepare ww ----
ww = np.empty((n_j, p_j))
for i in range(n_j):
    ww[i, 0] = np.unique(dat.presmean[dat.occ17 == i])
    ww[i, 1] = np.unique(dat.autmean[dat.occ17 == i])

In [808]:
# ---- Prepare xx ----
one = np.ones(n_i)
xx = dat[["educ", "age", "nonwhite"]]
xx = np.column_stack((one, xx))

# Simulate based on real data

In [809]:
n_employee = 100
n_employer = 4 # Unemployment 1; Professionals, Salaried 3; Craftsmen, Manufacturing 9; Service 14
employee_list = [None] * n_employee
employer_list = [None] * n_employer

In [810]:
my_ww = ww[np.array([0, 2, 8, 13]), :]
my_beta = np.array([[0, 0, 0],
                    [0.2, 0.15, 1],
                    [0.2, 0.1, -1],
                    [0.1, 0.3, 1]])
my_reserve = np.array([0, 8, 6, 12])
for i in range(n_employer):
    employer_list[i] = abm.Employer(w=my_ww[i, :], beta=my_beta[i, :], reserve_utility=my_reserve[i])

In [811]:
my_xx = xx[np.random.choice(xx.shape[0], size=n_employee, replace=False), :]
alpha = np.array([0.07, 1])
for i in range(n_employee):
    employee_list[i] = abm.Employee(x=my_xx[i, 1:], alpha=alpha)

In [812]:
xx[:, 1:].dot(my_beta[1:, :].T)

array([[ 8.15,  6.7 , 10.6 ],
       [ 7.55,  6.1 , 10.3 ],
       [ 7.5 ,  6.2 ,  9.6 ],
       ...,
       [ 8.15,  6.3 , 12.4 ],
       [ 7.75,  6.3 , 10.4 ],
       [ 7.15,  3.9 ,  9.7 ]])

In [813]:
my_model = abm.Model(n_employer=n_employer, n_employee=n_employee)
my_alpha, beta, ww, xx, choice, opp, observed_opp = \
    my_model.matching_process(employer_list=employer_list, employee_list=employee_list)

In [814]:
np.save("ww.npy", ww)
np.save("xx.npy", xx)
np.save("opp.npy", observed_opp)
np.save("choice.npy", choice)